In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
import sklearn
import pandas as pd
import scipy.stats as stats

#from keras import Sequential
#from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasRegressor

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression



In [2]:
neuron_data = {}

files = sorted(os.listdir('../neuron_logs/train_data'))

In [3]:
def reduce_to_statistics(activations, labels, debug=False):
    sorted_data = []
    for i in range(10): #hardcoded MOFO
        sorted_data += [[]]
    for i, a in zip(labels, activations):
        sorted_data[i] += [a]
    if debug:
        return(sorted_data)
    statistics = []
    for ar in sorted_data:
        curr_stats = stats.describe(ar)
        statistics += [curr_stats.mean, curr_stats.variance, curr_stats.skewness, curr_stats.kurtosis, curr_stats.minmax[0],
                       curr_stats.minmax[1], curr_stats.nobs]
        #print(statistics)
    return(statistics)

In [5]:
features, labels = [], []
target = 'usefulness_loss'
fin = 10
activations_no = 2000
for i, filename in enumerate(files[:]):
    with open(os.path.join('..', 'neuron_logs', 'train_data', filename), 'r') as f:
        print(filename)
        neuron_data = json.load(f)
    if i == 10:
        break
    for e in neuron_data.keys():
        if e == '0' or int(e) > fin:
            #print(e)
            continue
        for neuron in neuron_data[e]:
            if ' ' not in neuron:
                continue
            current_data = neuron_data[e][neuron]
            important_features = []
            important_features = current_data['activations'][:activations_no]
            important_features += reduce_to_statistics(current_data['activations'], neuron_data[e]['original_labels'])
            important_features += [current_data['depth']]
            important_features += [current_data['inverse_depth']]
            important_features += [current_data['width']]
            # important_features += [current_data['input_weights']]
            # important_features += [current_data['output_weights']]
            important_features += [current_data['reg_loss_in_layer']]
            important_features += [e]
            features += [important_features]
            #labels += [current_data[target]]
            labels += [current_data[target]]

output_20191013-181903.json
output_20191015-203714.json
output_20191015-215448.json
output_20191015-231129.json
output_20191016-002746.json
output_20191016-014510.json
output_20191016-030245.json
output_20191016-042123.json
output_20191016-053821.json
output_20191016-065412.json
output_20191016-081229.json


In [6]:
features = np.array(features)
labels = np.array(labels)
print(features.shape, labels.shape)

(12000, 2075) (12000,)


In [19]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim


import skorch
from skorch import NeuralNetClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from hunga_bunga import HungaBungaRegressor

ModuleNotFoundError: No module named 'regression'

In [20]:
import sys

sys.path.append('/home/levai/.miniconda3/envs/tensorflow/lib/python3.7/site-packages/hunga_bunga/')

from hunga_bunga import HungaBungaRegressor


In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

X_train = X_train.astype(np.float32)
#y_train = y_train.astype(np.int64)
X_test = X_test.astype(np.float32)
#y_test = y_test.astype(np.int64)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

testscaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [21]:
print(f"{X_train.shape} {X_test.shape} {y_train.shape} {y_test.shape}")
reg = HungaBungaRegressor().fit(X_train, y_train)


(9600, 2075) (2400, 2075) (9600,) (2400,)


  0%|          | 0/22 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
(reg.predict(X_test) - y_test)[:50]

array([-0.00993347,  0.5572921 , -0.0711695 ,  0.08415651, -0.02449381,
        0.02509594,  0.02790666,  0.13907182, -0.21004641,  0.12611961,
        0.09069145,  0.07778251, -0.01998508,  0.01863205,  0.14725924,
       -0.10609096, -0.05117428, -0.0210067 ,  0.10125113, -0.06807101,
        0.16649878,  0.14510095,  0.02564585,  0.00065041,  0.1015293 ,
        0.01946086, -0.24352109, -0.11581862,  0.03093755,  0.01394069,
       -0.01881349,  0.02935624,  0.11601043, -0.02912867,  0.04506731,
        0.09159672,  0.04174507,  0.31453049, -0.0747174 ,  0.17694163,
       -0.0207392 , -0.11683798,  0.03680825, -0.05585515, -0.47755909,
       -0.16412771, -0.04860127,  0.02574885, -0.05917156,  0.1730293 ])